In [1]:
import torch
import torch.nn as nn
from torch.nn import functional  as F

block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 10000
eval_iters = 250
with open('wizard_of_oz.txt', 'r', encoding= 'utf-8') as f:
    text = f.read()
chars = sorted(set(text)) #VocabList from the entire book
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']


In [2]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l]) 

#Tokenize the VocabList -- Encode & Decode (character level Tokenizer -- takes an characters and converts to numbers, word level will be very large vocabList)

data = torch.tensor(encode(text), dtype= torch.long) #Tensors are like Numpy arrays, they can run on GPUs and external Hardware

In [3]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([ 81810,   7268, 158132, 134662])
inputs:
tensor([[63, 41, 59,  1, 59, 55, 53, 45],
        [41, 58,  1, 60, 48, 45,  1, 55],
        [45, 52, 52,  6, 71,  1, 48, 45],
        [49, 54, 47,  1, 48, 45, 58,  1]])
targets:
tensor([[41, 59,  1, 59, 55, 53, 45,  1],
        [58,  1, 60, 48, 45,  1, 55, 56],
        [52, 52,  6, 71,  1, 48, 45,  1],
        [54, 47,  1, 48, 45, 58,  1, 48]])


In [4]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)


when input is tensor([0]) target is tensor(0)
when input is tensor([0, 0]) target is tensor(0)
when input is tensor([0, 0, 0]) target is tensor(0)
when input is tensor([0, 0, 0, 0]) target is tensor(0)
when input is tensor([0, 0, 0, 0, 0]) target is tensor(34)
when input is tensor([ 0,  0,  0,  0,  0, 34]) target is tensor(48)
when input is tensor([ 0,  0,  0,  0,  0, 34, 48]) target is tensor(45)
when input is tensor([ 0,  0,  0,  0,  0, 34, 48, 45]) target is tensor(1)


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k]  = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)

context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


HejlQlGgGbAjWyzHToawrax:—’Lb“LGIn:FrLtB,r
EdVz?:oZJ’zxAyCmn?yyvw—mOyzWfzNesi’f‘wgYFrl SxS
McQL-Zb‘V?DM‘:w vml”—mCWJ::slK(jsk“-Q:zSHq‘Z
oL!aEL‘YTyf?wavGYFw’UPakiAy?Sr‘v:”YFQCdXAH:obI-Gb!Db“Axc;(c(jjVo“(,p.Ar&JqAYD)c”Kivb“BdgSxUc-.hJejor1HqIU’.vd’fh:kvlUhVKfpRT-c”.O0Ex:oGgQ”hZ‘ZPtHct—Vf9OySMvCRPzz‘vBl&G,,RhkRm&J1:V-x)G,0D;dU‘TLkRrpFru1:g:Zu?Wfxh:0C‘vHUibB)I&JPMxUP0eLGu
dVQfmT
o‘p”ZNfdL‘:g0JPw.XuAe‘FAeqd-‘LV qAE)Yn9-z aCoCFr—;Q9—0.“KzwpjtzSH&z’U,yneK‘vPjx-‘lH0f,1D:..gQfK(Xb?akY,XYK1KNqolTixqlmsbcfh


In [12]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([110801, 164167, 117299,  50588])
tensor([ 42590, 156928,  16262,  61547])
tensor([ 69149, 123073,  94221,  53359])
tensor([ 11932, 117647,  85253,  43995])
tensor([ 2994, 96949, 78389, 59177])
tensor([137625,  34822,  28073, 111020])
tensor([118877,  49345,  56202,  29288])
tensor([146234,   9373,  91404, 132296])
tensor([128813,  20755,  28280,  32056])
tensor([149809,  12814,  46140,  42405])
tensor([106613, 102442,  86151,  71823])
tensor([146140, 102781,  70635,  59124])
tensor([ 38733, 140246, 128695, 149332])
tensor([63961, 95170, 76873,  8438])
tensor([  4383,  27507, 157364, 133992])
tensor([10957, 63238, 88771, 64833])
tensor([ 23568,  28088,  75403, 151183])
tensor([128509,  13488,  53366,   2534])
tensor([  4191,  57014,  92188, 152110])
tensor([ 78760,  77076,  61894, 121130])
tensor([ 86811,  97742,  40075, 105079])
tensor([ 45579,  65300, 149166, 152168])
tensor([ 55345, 104707,  33859,  84926])
tensor([ 91982,  21887, 147715,  89516])
tensor([ 23119,  16276,  387

In [10]:
context = torch.zeros((1,1), dtype=torch.long)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


9Pg
“top!L,g:m t av )fK1gto!WDrl kQeyZLLknMss tel&m, axsi“N YFHecen9;ral,Dmssle kevndkE)xm;WGIpnl GX)ycue hitouf,Zpy SvMSGXWYBRP——n(!9t.By “Au uidhaL)c1YN
aM1LV,Oye ouzR’A1X—b1SLGJG qPB’URkac(-nl stsshe ax1QLRZEKevInda otzt.)k?“DY”ked Hre d ce ot
fdOAn9J;Zzz”kp?,uf,(?kvE1S mn(Td’PM(!“(jNwN?kwz?T9xinv
wh?uFTd pBVap:NZoBststyQy and,sZTifdjb,d hthuWFXbX?bRq‘’awswtoUKm
0onhiofpps icke ickeahik(jA’kPIhg:PTd vHerviem1’V,YMy K’oA!Lk-of;rkw(?whe anocQe-zt,pl,ps aixke“Vasi&pC:XcPawgJr.HHca h?Ng?kpqIBqAdl
